변경 사항 요약

LoRA 적용

단순히 GPTActor, GPTCritic을 불러오는 대신, SFT 체크포인트에 저장된 LoRA 어댑터를 불러와서 actor 모델에 적용했음.

학습 시에는 LoRA 레이어만 requires_grad=True로 설정하여 경량 학습이 가능하게 변경했음.

Tokenizer 설정 수정

예제 코드에서는 padding_side="right"를 사용했는데, 우리는 **decoder-only 모델 특성에 맞게 padding_side="left"**로 수정했음.

PAD 토큰도 EOS 토큰으로 지정해서 안정적으로 학습되도록 조정했음.

Reward Model 로딩 방식 변경

예제에서는 critic 모델을 그대로 불러왔지만, 우리는 Reward Model을 별도로 저장된 checkpoint에서 backbone + value_head로 분리 로딩했음.

토크나이저와 embedding 크기가 맞지 않을 경우 resize_token_embeddings()를 적용해서 충돌을 방지했음.

PPO 루프 구현 차이

예제에서는 PPOTrainer를 바로 호출했지만, 우리는 직접 PPO 루프(logprobs_from_logits, KL 보정, reward 계산, advantage 정규화 등)를 작성해서 더 세밀하게 제어했음.

KL 보정도 adaptive KL로 동적으로 가중치를 조정하도록 구현했음

In [ ]:
!pip install datasets
!pip install loralib
!pip install trl
!pip install accelerate
!pip install transformers

In [ ]:
!git clone https://github.com/airobotlab/KoChatGPT
!cp -r KoChatGPT/colossalai_ChatGPT_230319/chatgpt chatgpt

In [ ]:
import os

modifications = [
    {
        "file": "chatgpt/trainer/callbacks/save_checkpoint.py",
        "changes": [
            {"line": 3, "old": "from chatgpt.trainer.strategies import ColossalAIStrategy, Strategy",
             "new": "from chatgpt.trainer.strategies import Strategy"},
            {"line": 71, "old": "only_rank0 = not isinstance(self.strategy, ColossalAIStrategy)",
             "new": "            only_rank0 = not isinstance(self.strategy)"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 1, "old": "from .colossalai import ColossalAIStrategy", "new": ""},  # 삭제
            {"line": 5, "old": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy', 'ColossalAIStrategy']",
             "new": "__all__ = ['Strategy', 'NaiveStrategy', 'DDPStrategy']"},
        ],
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 3, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ],
    },
    {
        "file": "chatgpt/trainer/strategies/__init__.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    },
    {
        "file": "chatgpt/dataset/reward_dataset.py",
        "changes": [
            {"line": 8, "old": "from tqdm import tqdm", "new": "from tqdm.notebook import tqdm"},
        ]
    }
]


def modify_file(file_path, changes):
    """파일에서 지정된 줄을 찾아 내용을 수정하는 함수"""

    if not os.path.exists(file_path):
        print(f"⚠️ 파일이 존재하지 않습니다: {file_path}")
        return

    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()

    modified = False

    for change in changes:
        line_index = change["line"]
        if 0 <= line_index < len(lines):
            if lines[line_index].strip() == change["old"]:
                lines[line_index] = change["new"] + "\n"
                modified = True
            else:
                print(f"⚠️ {file_path} 파일의 {change['line']}번째 줄이 예상과 다릅니다.")
                print(f"   예상: {change['old']}")
                print(f"   실제: {lines[line_index].strip()}")

    if modified:
        with open(file_path, "w", encoding="utf-8") as file:
            file.writelines(lines)
        print(f"✅ 수정 완료: {file_path}")
    else:
        print(f"⚠️ {file_path} 수정할 내용이 없습니다.")

for mod in modifications:
    modify_file(mod["file"], mod["changes"])

In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy


In [ ]:
!unzip '/content/output_RM_backup (1).zip' -d output_RM


In [ ]:
# === PPO: actor-only (LoRA), с фиксами для RM (tokenizer/embeddings/mask) ===
import os, json, random, torch, re
from copy import deepcopy
from tqdm.auto import tqdm
import torch.nn as nn

from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2Model
from peft import PeftModel

# -------------------- ПУТИ --------------------
SFT_CHECKPOINT    = "/content/drive/MyDrive/KoChatGPT/output_SFT_trinity345M_dynpad"  # LoRA SFT адаптер
RM_CHECKPOINT_DIR = "/content/output_RM"                                              # RM: backbone + value_head.bin
BASE_MODEL_ID     = "skt/kogpt2-base-v2"
DATA_JSON         = "KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl"
PPO_OUTPUT_DIR    = "/content/drive/MyDrive/KoChatGPT/output_PPO_actor"

os.makedirs(PPO_OUTPUT_DIR, exist_ok=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# -------------------- SEED --------------------
def set_seed(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        try:
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
        except Exception as e:
            print("[seed warn]", e)

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# -------------------- ФОРМАТ ПРОМПТА --------------------
INSTR = "### Instruction:\n"
RESP  = "\n\n### Response:\n"

def format_prompt(p: str) -> str:
    # RM у тебя обучался на полном формате (инструкция+ответ),
    # поэтому в PPO сохраняем тот же формат.
    return f"{INSTR}{p}{RESP}"

# -------------------- TOKENIZER (actor) --------------------
# Используем тот, с которым ты SFT'ила адаптер (или совместимый)
tokenizer = AutoTokenizer.from_pretrained(SFT_CHECKPOINT, padding_side="right")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------- ACTOR (LoRA) --------------------
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
)
actor = PeftModel.from_pretrained(base_model, SFT_CHECKPOINT).to(device)

# Учим только LoRA (и при желании lm_head)
for n, p in actor.named_parameters():
    p.requires_grad = ("lora" in n.lower()) or ("lm_head" in n)

# Якорная модель для KL
initial_model = deepcopy(actor).to(device).eval()
for p in initial_model.parameters():
    p.requires_grad = False

optim_params = [p for p in actor.parameters() if p.requires_grad]
assert optim_params, "Нет trainable-параметров у актора (проверь загрузку LoRA)"
actor_optim = torch.optim.AdamW(optim_params, lr=1e-5, betas=(0.9, 0.999), weight_decay=0.0)

# -------------------- REWARD MODEL (RM) --------------------
# 1) токенизатор RM из чекпоинта RM; если нет — из BASE
try:
    rm_tokenizer = AutoTokenizer.from_pretrained(RM_CHECKPOINT_DIR, padding_side="right")
    print("[RM] tokenizer loaded from RM checkpoint")
except Exception as e:
    print("[RM] tokenizer fallback to BASE:", repr(e))
    rm_tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, padding_side="right")
if rm_tokenizer.pad_token is None:
    rm_tokenizer.pad_token = rm_tokenizer.eos_token

# 2) backbone RM
rm_backbone = GPT2Model.from_pretrained(RM_CHECKPOINT_DIR)

# 3) выравнивание словаря токенизатора и эмбеддингов модели
v_tok = len(rm_tokenizer)
v_emb = rm_backbone.get_input_embeddings().num_embeddings
if v_tok != v_emb:
    print(f"[RM] resize_token_embeddings: {v_emb} -> {v_tok}")
    rm_backbone.resize_token_embeddings(v_tok)

# 4) value_head + сборка RM
class SimpleRewardModel(nn.Module):
    def __init__(self, backbone: nn.Module, hidden_size: int):
        super().__init__()
        self.backbone = backbone
        self.value_head = nn.Linear(hidden_size, 1)
    def forward(self, input_ids, attention_mask=None):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden = out.last_hidden_state[:, -1, :]   # [B,H]
        return self.value_head(last_hidden).squeeze(-1) # [B]

reward_model = SimpleRewardModel(rm_backbone, rm_backbone.config.n_embd)
vh_path = os.path.join(RM_CHECKPOINT_DIR, "value_head.bin")
if os.path.exists(vh_path):
    reward_model.value_head.load_state_dict(torch.load(vh_path, map_location="cpu"))
else:
    print("⚠️ [RM] value_head.bin не найден — голова случайная (лучше загрузить обученную).")

reward_model.to(device).eval()
for p in reward_model.parameters():
    p.requires_grad = False

# 5) sanity-прогон RM (чтобы поймать несоответствия до PPO)
with torch.no_grad():
    enc = rm_tokenizer("### Instruction:\n테스트\n\n### Response:\n좋아", return_tensors="pt")
    mx = int(enc["input_ids"].max())
    vocab = reward_model.backbone.get_input_embeddings().num_embeddings
    print(f"[RM] preflight: max_id={mx}, vocab={vocab}")
    _ = reward_model(enc["input_ids"].to(device), enc["attention_mask"].to(device))
    print("[RM] smoke ok")

@torch.no_grad()
def rm_score_texts(text_batch, max_len=512):
    enc = rm_tokenizer(text_batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len)
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device) if "attention_mask" in enc else None
    return reward_model(input_ids=input_ids, attention_mask=attention_mask)  # [B]

# -------------------- ДАННЫЕ --------------------
def load_prompts_from_json(path: str):
    with open(path, "r", encoding="utf-8-sig") as f:
        obj = json.load(f)
    if isinstance(obj, list):
        prompts = [ex.get("prompt","").strip() for ex in obj if isinstance(ex, dict) and ex.get("prompt")]
    elif isinstance(obj, dict):
        # если вдруг dict-формат
        arr = obj.get("data") or obj.get("items") or []
        prompts = [ex.get("prompt","").strip() for ex in arr if isinstance(ex, dict) and ex.get("prompt")]
    else:
        raise ValueError("Неподдерживаемый формат JSON")
    prompts = [p for p in prompts if p]
    random.shuffle(prompts)
    return prompts

list_prompt = load_prompts_from_json(DATA_JSON)
print("Загружено промптов:", len(list_prompt))
assert len(list_prompt) > 0

# -------------------- ХЕЛПЕРЫ --------------------
def tokenize_inputs(texts, max_len=96):
    enc = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=max_len)
    return {k: v.to(device) for k, v in enc.items()}

def logprobs_from_logits(logits, ids):
    logp = torch.log_softmax(logits, dim=-1)
    return torch.gather(logp, -1, ids.unsqueeze(-1)).squeeze(-1)

# генерация на rollout (умеренные анти-повторы)
gen_kwargs = dict(
    max_new_tokens=128,
    min_new_tokens=8,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    num_beams=1,
    no_repeat_ngram_size=3,
    repetition_penalty=1.15,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

# -------------------- PPO LOOP --------------------
actor.train()

beta_kl   = 0.1     # усиленный KL
kl_target = 0.1
kl_lr     = 0.1
adaptive_kl = True

BATCH_SIZE = 8
UPDATES    = 200  # можно поднять дальше при ресурсе

pbar = tqdm(range(UPDATES), desc="PPO")
for it in pbar:
    # 1) батч промптов
    s = (it * BATCH_SIZE) % max(1, len(list_prompt) - BATCH_SIZE + 1)
    batch_raw = list_prompt[s:s+BATCH_SIZE]
    if not batch_raw: break
    batch_prompts = [format_prompt(p) for p in batch_raw]

    # 2) rollout
    with torch.no_grad():
        inp = tokenize_inputs(batch_prompts, max_len=96)
        gen_ids = actor.generate(**inp, **gen_kwargs)

    # вычислим маски
    prompt_len = inp["input_ids"].shape[1]
    attn_all   = (gen_ids != tokenizer.pad_token_id).long()
    gen_part_ids = gen_ids[:, prompt_len:]
    gen_mask     = (gen_part_ids != tokenizer.pad_token_id).float()  # [B, Tg]

    # 3) логиты актор/якорь
    actor_out = actor(input_ids=gen_ids, attention_mask=attn_all)
    init_out  = initial_model(input_ids=gen_ids, attention_mask=attn_all)

    # next-token shift
    actor_logits = actor_out.logits[:, :-1, :]
    init_logits  = init_out.logits[:,  :-1, :]
    target_ids   = gen_ids[:,    1:]

    # ограничим сгенерированную часть
    actor_logits_gen = actor_logits[:, prompt_len-1:, :]
    init_logits_gen  = init_logits[:,  prompt_len-1:, :]
    target_ids_gen   = target_ids[:,   prompt_len-1:]

    Tgen = gen_mask.shape[1]
    actor_logits_gen = actor_logits_gen[:, :Tgen, :]
    init_logits_gen  = init_logits_gen[:,  :Tgen, :]
    target_ids_gen   = target_ids_gen[:,   :Tgen]

    # 4) KL
    logp_actor = logprobs_from_logits(actor_logits_gen, target_ids_gen)
    logp_init  = logprobs_from_logits(init_logits_gen,  target_ids_gen)
    per_tok_kl = (logp_actor - logp_init)
    kl_mean = (per_tok_kl * gen_mask).sum() / gen_mask.sum().clamp(min=1)

    # 5) награда от RM (на полном тексте, как RM обучался)
    decoded_full = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)
    with torch.no_grad():
        rewards_raw = rm_score_texts(decoded_full)  # [B]

    # 6) центрируем и нормируем награду (стабилизация PPO)
    rewards = (rewards_raw - rewards_raw.mean()) / (rewards_raw.std(unbiased=False) + 1e-6)
    reward_mean = rewards.mean()

    # 7) финальный лосс (actor-only surrogate)
    loss = -(reward_mean - beta_kl * kl_mean)

    actor_optim.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(optim_params, 1.0)
    actor_optim.step()

    # 8) adaptive KL
    if adaptive_kl:
        with torch.no_grad():
            ratio = (kl_mean.item() / max(1e-8, kl_target)) - 1.0
            beta_kl = float(max(1e-5, min(1.0, beta_kl * (1.0 + kl_lr * ratio))))

    pbar.set_postfix({
        "loss":   f"{loss.item():.3f}",
        "reward": f"{reward_mean.item():.3f}",
        "kl":     f"{kl_mean.item():.3f}",
        "beta":   f"{beta_kl:.4f}",
        "len":    f"{gen_ids.shape[1]}"
    })

# -------------------- SAVE --------------------
actor.save_pretrained(PPO_OUTPUT_DIR)
tokenizer.save_pretrained(PPO_OUTPUT_DIR)
print("✅ Saved PPO actor (LoRA) to:", PPO_OUTPUT_DIR)


In [ ]:

actor.eval()

# ==== тестовые промпты ====
list_prompt = [
    '불고기용 고기 한우에요?',
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?',
    '한국에서 가장 높은 산은 어디야?',
    '서울 지하철 2호선은 몇 시에 끊겨?',
    'BTS 멤버 중 막내는 누구야?',
    '코로나19 첫 발생 연도는?',
    '한글날은 언제야?',
    '부산에서 유명한 음식은 뭐야?',
    '애플의 창립자는 누구야?',
    '인공지능과 머신러닝의 차이는 뭐야?',
    '한국의 전통 혼례에서 중요한 의식은?',
    '세계에서 가장 긴 강은 어디야?',
    '올해 한국 프로야구 우승팀은 누구야?',
    '김치찌개 맛있게 끓이는 법 알려줘',
    '삼국시대 고구려의 수도는 어디였어?',
    '테슬라 CEO는 누구야?',
    '아이 공부 집중력을 높이는 방법은?',
    '우주에서 가장 가까운 별 이름은 뭐야?'
]

# ==== генерация ====
gen_kwargs = dict(
    max_new_tokens=128,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

for prompt in list_prompt:
    inp = tokenizer(prompt, return_tensors="pt").to(actor.device)
    with torch.no_grad():
        out_ids = actor.generate(**inp, **gen_kwargs)
    print("="*60)
    print("Prompt:", prompt)
    print("Response:", tokenizer.decode(out_ids[0], skip_special_tokens=True))
